In [0]:
import pandas as pd
from google.colab import files

In [0]:
uploaded = files.upload()

Saving News_category_Train.xlsx to News_category_Train.xlsx


In [0]:
uploaded1 = files.upload()

Saving News_category_Test.xlsx to News_category_Test.xlsx


In [0]:
train_data = pd.read_excel("News_category_Train.xlsx", delimiter='\t')

In [0]:
test_data = pd.read_excel("News_category_Test.xlsx", delimiter = '\t')

In [0]:
train_data.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [0]:
test_data.head()

,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...


In [0]:
print('Training Dataset Analysis\n')
train_data.info()
print('\nTrain dataset shape',train_data.shape)
print('\nUnique value of Sections',train_data['SECTION'].unique())

Training Dataset Analysis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 2 columns):
STORY      7628 non-null object
SECTION    7628 non-null int64
dtypes: int64(1), object(1)
memory usage: 119.3+ KB

Train dataset shape (7628, 2)

Unique value of Sections [3 0 1 2]


In [0]:
print('Test Dataset Analysis\n')
test_data.info()
print('\nTest dataset shape',test_data.shape)

Test Dataset Analysis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 1 columns):
STORY    2748 non-null object
dtypes: object(1)
memory usage: 21.5+ KB

Test dataset shape (2748, 1)


In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [0]:
def clean_function(data):
  story = re.sub('[^a-zA-Z]',' ',data)
  story = story.lower()
  story = story.split()
  story = [word for word in story if not word in set(stopwords.words('english'))]
  #story = [ps.stem(word) for word in story if not word in set(stopwords.words('english'))]
  data = ' '.join(story)
  return data
  

In [0]:
data = 'I loved this Place.'

In [0]:
clean_function(data)

'loved place'

In [0]:
train_data["New_Cleaned_story"] = train_data["STORY"].apply(clean_function)

In [0]:
train_data.head()

,STORY,SECTION,New_Cleaned_story
0,But the most painful was the huge reversal in ...,3,painful huge reversal fee income unheard among...
1,How formidable is the opposition alliance amon...,0,formidable opposition alliance among congress ...
2,Most Asian currencies were trading lower today...,3,asian currencies trading lower today south kor...
3,"If you want to answer any question, click on ‘...",1,want answer question click answer clicking ans...
4,"In global markets, gold prices edged up today ...",3,global markets gold prices edged today disappo...


In [0]:
train_data.shape

(7628, 3)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train = cv.fit_transform(train_data['New_Cleaned_story']).toarray()

In [0]:
X_train.shape

(7628, 30831)

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
new = tfidf.fit_transform(X_train)

In [0]:
new.shape

(7628, 30831)

In [0]:
test_data['New_Clean_Test'] = test_data['STORY'].apply(clean_function)
test_data[['STORY','New_Clean_Test']].head()

,STORY,New_Clean_Test
0,2019 will see gadgets like gaming smartphones ...,see gadgets like gaming smartphones wearable m...
1,It has also unleashed a wave of changes in the...,also unleashed wave changes mcu make sure futu...
2,It can be confusing to pick the right smartpho...,confusing pick right smartphone segregated top...
3,The mobile application is integrated with a da...,mobile application integrated dashboard confir...
4,We have rounded up some of the gadgets that sh...,rounded gadgets showed left indelible mark con...


In [0]:
y = train_data['SECTION']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new,y, test_size=0.2, random_state=42)

In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred = classifier.predict(X_test)

In [0]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape, y_pred.shape

((6102, 30831), (1526, 30831), (1526,), (6102,), (1526,))

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.916775884665793

In [0]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,y_train)
y_ppred = nb.predict(X_test)

In [0]:
accuracy_score(y_test, y_ppred)

0.9475753604193972

Sequentially apply a list of transforms and a final estimator

In [0]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(),TfidfTransformer(),MultinomialNB())
pipe.fit(train_data['New_Cleaned_story'], train_data['SECTION'])

test_pred = pipe.predict(test_data['New_Clean_Test'])
print(test_pred)

[1 2 1 ... 1 0 1]


In [0]:
t ='This is such a secenic beauty.'
t = [t]
pred = pipe.predict(t)

In [0]:
pred

array([1])